In [19]:
import sys
import json
sys.path.append('..')
from common import *
from analysis.model_performances import *
from questions_construction.questions import *
from copy import deepcopy
from helpers import *
import pandas as pd
import scipy
            
TO_PRETTY = {"fluent_tracking": 'Fluent Tracking', 
             "state_tracking": 'State Tracking', 
             "action_executability": 'Action Executability', 
             "effects": 'Effects of Act.', 
             "numerical_reasoning": 'Numerical RAC', 
             "composite": 'Composite', 
             "ALL_CATEGORIES": 'Average'}

In [20]:
substitution = WITHOUT_RANDOM_SUB
ramification = WITHOUT_RAMIFICATIONS
prompt_type = ZERO_SHOT_PROMPT_KEY #FEW_SHOT_3_PROMPT_KEY #
models = ['gpt-4o', 'llama_8b','llama_70b', 'llama_8b.finetuned_tf']

data_all = []
for model_name in models:
    model_results_dir = f'{CODE_PATH}/results/{ramification}/{prompt_type}/{model_name}.jsonl'
    model_results = open_jsonl(model_results_dir)
    data_all.extend(model_results)

In [21]:
data_all[0]

{'question_id': 'c78f1141-2951-4870-b971-823852089973',
 'lebel': 'False',
 'prompt': '[DOMAIN DESCRIPTION]\nA block can be picked up only if it is clear, on the table, and the hand is empty. Once picked up, the block is no longer on the table or clear, and the hand is no longer empty. To put a block down, it must be held, and doing so places it on the table, making it clear and leaving the hand becomes empty. A block can be stacked on another if it is held and the other block is clear, which results in the stacked block becoming clear and the second block no longer clear, with the hand becoming empty. Conversely, unstacking requires the hand to be empty and the first block to be clear and on top of the second block. After unstacking, the second block becomes clear, while the first block is held, no longer clear, and no longer on top of the second block, leaving the hand not empty.\n\n[INITIAL CONDITIONS]\nInitially, Block b1 is stacked on block b7, block b2 has no blocks on top of it,

In [24]:
def to_df_by_domains(data_all):

    # index = []
    data_for_df = []   
    for plan_length in [1, 10, 19]:
        for question_category in QUESTION_CATEGORIES+[ALL_QUESTION_CATEGORIES_KEY]:
            data_columns = {'Act. Seq.': plan_length, 'Ques. Categories': TO_PRETTY[question_category]}
            for domain in ['blocksworld', 'depots', 'driverlog', 'grippers', 'mystery', 'satellite', 'spanner', 'visitall']:
                if question_category == ALL_QUESTION_CATEGORIES_KEY:
                    data_selected = data_all
                else:
                    data_selected = [d for d in data_all if d[OUT_OBJ_QUESTION_CATEGORY] == question_category]
                scores = [d[IS_RESPONSE_CORRECT_KEY] == TRUE_ANSWER for d in data_selected if d[OUT_OBJ_DOMAIN_NAME] == domain and d['plan_length'] == plan_length and d[IS_RESPONSE_CORRECT_KEY] in (TRUE_ANSWER, FALSE_ANSWER)]
                if scores:
                    final_res = (np.mean(scores), scipy.stats.sem(scores))
                else:
                    final_res = (None, None)
                final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
                final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
                data_columns[domain.capitalize()] = final_res
            data_for_df.append(data_columns)
    return pd.DataFrame(data_for_df)

# Subs and Ramfications

In [25]:
model = models[1]
df = to_df_by_domains(data_all)
df
        
# caption_nl = f'performance of {model_name}, {prompt_type}, pl-{plan_length}'.replace('_', ' ')
# save_key = f'{model_name}.{prompt_type}.{plan_length}'
# 
# latex_table_all = to_latex_table(df, caption_nl, label=save_key, index=False)
# with open(os.path.join(save_dir, f'{save_key}.tex'), 'w') as f:
#     f.write(latex_table_all)

,Act. Seq.,Ques. Categories,Blocksworld,Depots,Driverlog,Grippers,Mystery,Satellite,Spanner,Visitall
0,1,Fluent Tracking,${91.3}_{4.2}$,${72.22}_{6.15}$,${64.44}_{7.22}$,${91.55}_{3.32}$,${84.44}_{5.46}$,${75.0}_{5.64}$,${77.61}_{5.13}$,${78.18}_{5.62}$
1,1,State Tracking,${70.21}_{6.74}$,${81.25}_{5.69}$,${53.85}_{9.97}$,${92.0}_{3.88}$,${78.95}_{6.7}$,${60.47}_{7.54}$,${68.63}_{6.56}$,${91.67}_{5.76}$
2,1,Action Executability,${60.32}_{6.21}$,${78.95}_{5.45}$,${68.33}_{6.06}$,${79.66}_{5.29}$,${62.71}_{6.35}$,${70.77}_{5.69}$,${72.31}_{5.59}$,${63.27}_{6.96}$
3,1,Effects of Act.,${75.68}_{5.02}$,${69.23}_{6.46}$,${73.91}_{6.55}$,${73.08}_{5.05}$,${83.72}_{5.7}$,${61.54}_{6.81}$,${70.21}_{6.74}$,${80.0}_{5.71}$
4,1,Numerical RAC,${49.06}_{6.93}$,${41.03}_{7.98}$,${31.25}_{6.76}$,${45.45}_{7.59}$,${30.23}_{7.09}$,${35.48}_{8.74}$,${37.5}_{6.1}$,${34.78}_{5.78}$
5,1,Composite,${60.26}_{4.0}$,${52.55}_{4.28}$,${61.97}_{4.09}$,${62.2}_{3.8}$,${58.5}_{4.08}$,${56.71}_{3.88}$,${61.01}_{3.88}$,${62.02}_{4.29}$
6,1,Average,${65.9}_{2.28}$,${63.82}_{2.45}$,${60.22}_{2.56}$,${72.32}_{2.07}$,${64.0}_{2.48}$,${60.96}_{2.4}$,${63.58}_{2.26}$,${63.83}_{2.48}$
7,10,Fluent Tracking,${88.24}_{8.05}$,${71.43}_{7.06}$,${82.69}_{5.3}$,${88.41}_{3.88}$,${61.11}_{8.24}$,${67.14}_{5.65}$,${69.44}_{5.47}$,${80.33}_{5.13}$
8,10,State Tracking,${84.62}_{5.85}$,${48.28}_{9.44}$,${73.53}_{7.68}$,${75.0}_{6.6}$,${58.62}_{9.31}$,${71.43}_{7.06}$,${72.55}_{6.31}$,${64.0}_{9.8}$
9,10,Action Executability,${50.85}_{6.56}$,${66.67}_{7.36}$,${64.15}_{6.65}$,${48.44}_{6.3}$,${53.85}_{8.09}$,${59.62}_{6.87}$,${61.02}_{6.4}$,${64.81}_{6.56}$


In [26]:
latex_table_all = to_latex_table(df, 'some caption', index=False)
print(latex_table_all)


\begin{table}[h!]
\begin{adjustbox}{width=\textwidth,center}
\begin{tabular}{rlllllllll}
\toprule
 Act. Seq. &     Ques. Categories &      Blocksworld &           Depots &        Driverlog &          Grippers &          Mystery &        Satellite &          Spanner &         Visitall \\
\midrule
         1 &      Fluent Tracking &   ${91.3}_{4.2}$ & ${72.22}_{6.15}$ & ${64.44}_{7.22}$ &  ${91.55}_{3.32}$ & ${84.44}_{5.46}$ &  ${75.0}_{5.64}$ & ${77.61}_{5.13}$ & ${78.18}_{5.62}$ \\
         1 &       State Tracking & ${70.21}_{6.74}$ & ${81.25}_{5.69}$ & ${53.85}_{9.97}$ &   ${92.0}_{3.88}$ &  ${78.95}_{6.7}$ & ${60.47}_{7.54}$ & ${68.63}_{6.56}$ & ${91.67}_{5.76}$ \\
         1 & Action Executability & ${60.32}_{6.21}$ & ${78.95}_{5.45}$ & ${68.33}_{6.06}$ &  ${79.66}_{5.29}$ & ${62.71}_{6.35}$ & ${70.77}_{5.69}$ & ${72.31}_{5.59}$ & ${63.27}_{6.96}$ \\
         1 &      Effects of Act. & ${75.68}_{5.02}$ & ${69.23}_{6.46}$ & ${73.91}_{6.55}$ &  ${73.08}_{5.05}$ &  ${83.72}_{5.7}$ & 

/Users/paveldolin/dev/research/completed/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:45: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=index, formatters={"name": str.upper}, float_format="{:.2f}".format)


In [80]:
df_pos_neg = to_df_by_sub_ram_neg_pos(data_all, model_name, prompt_type, plan_length=plan_length)
df_pos_neg
        
# caption_nl = f'performance of {model_name}, {prompt_type}, pl-{plan_length}'.replace('_', ' ')
# save_key = f'{model_name}.{prompt_type}.{plan_length}'
# 
# latex_table_all = to_latex_table(df, caption_nl, label=save_key, index=False)
# with open(os.path.join(save_dir, f'{save_key}.tex'), 'w') as f:
#     f.write(latex_table_all)

NameError: name 'OUT_OBJ_IS_POS_FLUENT_QUESTION' is not defined

# By Models

In [45]:
score_type=ACCURACY_SCORE_KEY, 
question_category = ALL_QUESTION_CATEGORIES_KEY,
domain = ALL_DOMAINS_KEY,
answer_type=TRUE_FALSE_ANSWER_TYPE,
plan_length=1

data_for_df = []    
for fluent_type in FLUENT_TYPES_LIST:
    # index.append()
    data_columns = {}
    data_columns['fluent type'] = TO_PRETTY[fluent_type]
    for model in models:
        data = filter_multi_selector_modified(data_all, ramification, model, prompt_type, answer_type, substitution, plan_length, [(OUT_OBJ_FLUENT_TYPE, {fluent_type})])
        print(len(data))
        stats = TrueFalseStatsCustom(data, plan_length, question_category, ramification, model, prompt_type, domain, substitution, score_type=score_type)
        res_obj = stats.compute()
        if res_obj:
            mean = res_obj['result']
            sem = None
            if res_obj['result_other']:
                sem = res_obj['result_other']['sem']
            final_res = (mean, sem)
        else:
            final_res = (None, None)
        final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
        final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
        # print(final_res)
        data_columns[TO_PRETTY.get(model, model)] = final_res
    data_for_df.append(data_columns)

0
0
0
0
0
0
0
0
0
0
0
0


In [64]:
data = filter_multi_selector_modified(data_all, ramification, 'llama_8b', prompt_type, 'free_answer', substitution, 1, [(OUT_OBJ_FLUENT_TYPE, {fluent_type})])

In [65]:
data

[]

In [29]:
prompt_type

'zero_shot'

In [54]:
prompt_type

'zero_shot'

In [55]:
substitution

'without_random_sub'

In [56]:
data_all[0]

{'question_id': '784689cc-3f7c-4a22-824a-ffc0e4c90483',
 'domain_name': 'logistics',
 'instance_id': 'Instance_2',
 'question_category': 'composite',
 'question_name': 'iter_6_question_2',
 'fluent_type': 'all_fluents',
 'answer_type': 'free_answer',
 'question': 'Given the initial condition, the following actions are planned to be performed: at airport l1_1, package p2 is loaded in truck t1, at airport l1_1, package p1 is loaded in truck t1, truck t1 is driven to airport l1_0 from airport l1_1 in city c1, in city c1, truck t0 is driven from airports l1_1 to l0_1, from truck t1 package p1 is unloaded at airport l1_0, in city c0, truck t0 is driven from airports l0_1 to l0_0, at airport l0_0, package p3 is loaded in airplane a0, airplane a0 flies from airports l0_0 to l1_0, from airplane a0 package p3 is unloaded at airport l1_0 and package p3 is loaded in truck t1 at airport l1_0 to reach the current state. What are the valid properties of the state that do not involve negations for p1